# Exploració de les categories `currentUse` dels edificis de Barcelona

Aquest notebook carrega el fitxer `barcelona_buildings.geojson`,
mostra les categories del camp `currentUse`, les agrupa i en fa una visualització ràpida.

Assegura't que el fitxer **`barcelona_buildings.geojson`** està al mateix directori que aquest notebook.

In [ ]:
# Instal·la dependències si cal (comenta-ho si ja les tens)
# %pip install geopandas matplotlib

import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd

plt.rcParams['figure.figsize'] = (10, 6)

print('Llibreries carregades.')

In [ ]:
# Carrega el GeoJSON
path = 'barcelona_buildings.geojson'
gdf = gpd.read_file(path)
print(f'Files carregades: {len(gdf)}')
gdf.head()

In [ ]:
# Comprovem que el camp currentUse existeix
if 'currentUse' not in gdf.columns:
    raise KeyError("El camp 'currentUse' no existeix al fitxer. Comprova el nom exacte de la columna.")

gdf['currentUse'].head()

In [ ]:
# Neteja bàsica del camp currentUse
current = gdf['currentUse'].astype(str).str.strip()
current = current.replace({'': None, 'nan': None, 'None': None})
gdf['currentUse_clean'] = current

print('Valors únics (mostra fins a 50):')
pd.Series(gdf['currentUse_clean'].unique()).head(50)

In [ ]:
# Comptar freqüències per categoria
freq = gdf['currentUse_clean'].value_counts(dropna=True).reset_index()
freq.columns = ['currentUse', 'count']

freq.head(20)

In [ ]:
# Gràfic: Top 20 categories de currentUse
top_n = 20
top_freq = freq.head(top_n)

plt.figure()
plt.barh(top_freq['currentUse'], top_freq['count'])
plt.gca().invert_yaxis()
plt.xlabel('Nombre d\'edificis')
plt.title(f'Top {top_n} categories de currentUse')
plt.tight_layout()
plt.show()

In [ ]:
# Agrupació opcional de categories similars
# Exemple: agrupar per paraules clau (modifica segons les teves dades)

def map_group(value: str) -> str:
    if value is None:
        return 'Sense dades'
    v = value.lower()
    if 'resid' in v or 'habitatge' in v:
        return 'Residencial'
    if 'office' in v or 'oficina' in v or 'administr' in v:
        return 'Oficines / Administració'
    if 'commer' in v or 'retail' in v or 'botiga' in v:
        return 'Comercial'
    if 'industrial' in v or 'factory' in v:
        return 'Industrial'
    if 'educat' in v or 'school' in v or 'univer' in v:
        return 'Educatiu'
    if 'health' in v or 'hospital' in v or 'clinic' in v:
        return 'Salut'
    if 'hotel' in v or 'hostel' in v or 'allotjament' in v:
        return 'Hotel / Allotjament'
    if 'sport' in v or 'esport' in v:
        return 'Esportiu'
    if 'culture' in v or 'cultura' in v or 'museum' in v:
        return 'Cultural'
    return 'Altres'

gdf['currentUse_group'] = gdf['currentUse_clean'].apply(map_group)

group_freq = gdf['currentUse_group'].value_counts().reset_index()
group_freq.columns = ['group', 'count']
group_freq

In [ ]:
# Gràfic de les categories agrupades
plt.figure()
plt.barh(group_freq['group'], group_freq['count'])
plt.gca().invert_yaxis()
plt.xlabel('Nombre d\'edificis')
plt.title('Edificis per categoria agrupada (currentUse_group)')
plt.tight_layout()
plt.show()

In [ ]:
# Vista ràpida al mapa: pintar els edificis segons la categoria agrupada
# (això pot trigar si hi ha molts edificis)

ax = gdf.plot(column='currentUse_group', legend=True, figsize=(10, 10))
ax.set_title('Mapa d\'ús actual dels edificis (agrupat)')
ax.set_axis_off()
plt.show()